## add filtering def

In [1]:
import sys
import numpy as np
import pickle
import os
from matplotlib import pyplot as plt 
import pandas as pd

In [2]:
os.chdir("/home/thlamp/tcga/python_results")

In [3]:
def progress(s):
    sys.stdout.write("%s" % (str(s)))
    sys.stdout.flush()


def message(s):
    sys.stdout.write("%s\n" % (str(s)))
    sys.stdout.flush()

In [4]:
global FEATURE_VECTOR_FILENAME
FEATURE_VECTOR_FILENAME = "/home/thlamp/tcga/bladder_results/raw_data_integrated_matrix.txt"
global Prefix
Prefix = ""

In [5]:
global FEATURE_VECTOR_FILENAME
FEATURE_VECTOR_FILENAME = "/home/thlamp/tcga/bladder_results/normalized_data_integrated_matrix.txt"
global Prefix
Prefix = ""

In [103]:
def initializeFeatureMatrices(bResetFiles=False, bPostProcessing=True, bNormalize=True,
                              bNormalizeLog2Scale=True):
    """
    Initializes the case/instance feature matrices, also creating intermediate files for faster startup.

    :param bResetFiles: If True, then reset/recalculate intermediate files. Default: False.
    :param bPostProcessing: If True, then apply post-processing to remove NaNs, etc. Default: True.
    :param bNormalize: If True, then apply normalization to the initial data. Default: True.
    :param bNormalizeLog2Scale: If True, then apply log2 scaling after normalization to the initial data. Default: True.
    :return: The initial feature matrix of the cases/instances.
    """

    message("Opening files...")

    try:
        if bResetFiles:
            raise Exception("User requested file reset...")
        message("Trying to load saved data...")

        # Apply np.load hack
        ###################
        # save np.load
        np_load_old = np.load ##return the input array from a disk file with npy extension(.npy)

        # modify the default parameters of np.load
        np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)##lambda *a, **k: orizetai to lamda function and *a, **k function parameters
        ##np.load is a function provided by the NumPy library, typically used for loading data from saved files
        ## BUT np.load = ... assigns a new behavior to the np.load function. This assignment changes how the np.load function works for the duration of the current scope or context in which it's defined
        
        ##lambda *a, **k: defines an anonymous function (a lambda function) that takes any number of positional arguments as a tuple a and any number of keyword arguments as a dictionary k. This lambda function is like a wrapper around the original np.load function.
        ## call load_data with allow_pickle implicitly set to true
        
        ##allow_pickle=True: An additional keyword argument specifying that pickled objects are allowed to be loaded.
        
        datafile = np.load(Prefix + "patientAndControlData.mat.npy")
        labelfile = np.load(Prefix + "patientAndControlDataLabels.mat.npy")

        # restore np.load for future normal usage
        np.load = np_load_old ##orizei to np.load sthn default, arxikh leitourgia
        ####################

        clinicalfile = loadTumorStage()##epistrefei A matrix indicating the tumor stage per case/instance
        message("Trying to load saved data... Done.")
    except Exception as eCur:
        message("Trying to load saved data... Failed:\n%s" % (str(eCur)))
        message("Trying to load saved data from txt...")
        fControl = open(FEATURE_VECTOR_FILENAME, "r")
        message("Loading labels and ids...")
        # labelfile, should have stored tumor_stage or labels?       
        
        labelfile = np.genfromtxt(fControl, skip_header=1, usecols=(0, 100472),
                                  missing_values=['NA', "na", '-', '--', 'n/a'],
                                  dtype=np.dtype("object"), delimiter=' ').astype(str)
        ##numpy.genfromtxt function to read data from a file. This function is commonly used to load data from text files into a NumPy array.
        ##dtype=np.dtype("object"): This sets the data type for the resulting NumPy array to "object," which is a generic data type that can hold any type of data
        
        #+ removes " from first column 
        labelfile[:, 0] = np.char.replace(labelfile[:, 0], '"', '')
        
        message("This is the label file...")
        message(labelfile)
        
        message("Splitting features, this is the size of labelfile")
        message(np.shape(labelfile))

        message("Loading labels and ids... Done.")
        #---substitute loadTumorStage with clinicalfile = np.genfromtxt ...
        #clinicalfile = loadTumorStage()
        # Reset the file cursor to the beginning
        
        clinicalfile = loadTumorStage()
        
        fControl.close()

        datafile = loadPatientAndControlData()##return: the patient and control feature data file as a matrix
        message("Trying to load saved data from txt... Done.")
        print(datafile[:5,:5])
        # Saving
        saveLoadedData(datafile, labelfile)##Saves intermediate data and label file matrices for quick loading.

    message("Opening files... Done.")
	
    # Split feature set to features/target field
    mFeatures, vClass, sampleIDs = splitFeatures(clinicalfile, datafile, labelfile)##return: A tuple of the form (matrix of features, matrix of labels, sample ids)
    
    mControlFeatureMatrix = getControlFeatureMatrix(mFeatures, vClass)#return: The subset of the data matrix, reflecting only control cases/instances.
    message("1 .This is the shape of the control matrix:")
    message(np.shape(mControlFeatureMatrix))


    # the new bPostProcessing removes columns from mFeatures and mControlFeatureMatrix
    if bPostProcessing:
        mFeatures = postProcessFeatures(mFeatures, mControlFeatureMatrix)
        ##return: The post-processed matrix, without NaNs.

    # Update control matrix, taking into account postprocessed data
    mControlFeatureMatrix = getControlFeatureMatrix(mFeatures, vClass)
    ##Update control matrix, taking into account postprocessed data, an exei tre3ei to bPostProcessing dn 8a exei NaNs


    message("2 .This is the shape of the control matrix:")
    message(np.shape(mControlFeatureMatrix))

    if bNormalize:
        mFeatures = normalizeDataByControl(mFeatures, mControlFeatureMatrix, bNormalizeLog2Scale)
        ##return: The normalized and - possibly - log scaled version of the input feature matrix.
    return mFeatures, vClass, sampleIDs

In [6]:
def loadTumorStage():
    """
    Gets tumor stage data from clinical data file.
    :return: A matrix indicating the tumor stage per case/instance.
    """
    # Tumor stage
    message("Loading tumor stage...")
    fControl = open(FEATURE_VECTOR_FILENAME, "r")
    # While loading stage, also convert string to integer
    clinicalfile = np.genfromtxt(fControl, skip_header=1, usecols=(0, 100473),
                                  missing_values=['NA', "na", '-', '--', 'n/a'],
                                  dtype=np.dtype("object"), delimiter=' ').astype(str)
    ##np.genfromtxt: This is a NumPy function used to load data from text files, including delimited files. It's a flexible function that can handle a variety of file formats.
    ##dtype=np.dtype("object"): This specifies the data type for the loaded data. The data type is set to "object," which is a flexible data type that can hold a variety of data types, including strings.
    clinicalfile[:, 0] = np.char.replace(clinicalfile[:, 0], '"', '')
    fControl.close()
    message("Loading tumor stage... Done.")
    message("This is the clinical file...")
    message(clinicalfile)
    message("These are the dimensions of the clinical file")
    message(np.shape(clinicalfile))
    return clinicalfile

In [7]:
def loadPatientAndControlData():
    """
    Loads and returns the serialized patient and control feature data file as a matrix.
    :return: the patient and control feature data file as a matrix
    """
    message("Loading features...")
    fControl = open(FEATURE_VECTOR_FILENAME, "r")
    datafile = np.genfromtxt(fControl, skip_header=1, usecols=range(1, 100472),
                             missing_values=['NA', "na", '-', '--', 'n/a'], delimiter=" ",
                             dtype=np.dtype("float")
                             )
    ##numpy.genfromtxt function to read data from a file. This function is commonly used to load data from text files into a NumPy array.
    ##dtype=np.dtype("float"): This sets the data type for the resulting NumPy array to float
    fControl.close()

    message("This is the datafile...")
    message(datafile)
    message("Loading features... Done.")
    return datafile

In [8]:
def saveLoadedData(datafile, labelfile):
    """
    Saves intermediate data and label file matrices for quick loading.
    :param datafile: The matrix containing the feature data.
    :param labelfile: The matrix containing the label data.
    """
    message("Saving data in dir..." + os.getcwd())
    np.save(Prefix + "patientAndControlData.mat.npy", datafile)
    np.save(Prefix + "patientAndControlDataLabels.mat.npy", labelfile)
    message("Saving data... Done.")

In [9]:
def splitFeatures(clinicalfile, datafile, labelfile): 
    """
    Extracts class and instance info, returning them as separate matrices, where rows correspond to the same
    case/instance.

    :param clinicalfile: The file with the clinical info.
    :param datafile: The matrix containing the full feature data from the corresponding file.
    :param labelfile: The matrix containing  the full label data from the corresponding file.
    :return: A tuple of the form (matrix of features, matrix of labels)
    Chris update: :return: A tuple of the form (matrix of features, matrix of labels, sample ids)
    """
    message("Splitting features...")
    message("Number of features: %d"%(np.size(datafile, 1)))
    message("This is the label file:")
    message(labelfile)
    message("This is the shape of the labelfile: %s" % (str(np.shape(labelfile))))
    mFeaturesOnly = datafile[:, :]##datafile = the patient and control feature data file as a matrix
    # Create matrix with extra column (to add tumor stage)
    iFeatCount = np.shape(mFeaturesOnly)[1] + 1
    ##np.shape(mFeaturesOnly)[1] After getting the shape, you are accessing the second element of the tuple, which corresponds to the number of columns in the array
    ##number of columns + 1
    # DEBUG LINES
    message("Label file rows: %d\tFeature file rows: %d"%(np.shape(labelfile)[0], np.shape(mFeaturesOnly)[0]))
    #############

    mFeatures = np.zeros((np.shape(mFeaturesOnly)[0], iFeatCount))
    mFeatures[:, :-1] = mFeaturesOnly
    mFeatures[:, iFeatCount - 1] = np.nan##last column of the NumPy array mFeatures to np.nan
    
    #---
    #tumorStageToInt = np.vectorize(convertTumorType)##Converts tumor stages to float numbers, based on an index of classes.
    choicelist = clinicalfile[:, 1].astype(float)
    ## clinicalfile[:, 1]: This code extracts the entire second column (column with index 1) from the NumPy array clinicalfile
    ## choicelist contains the float number representations of tumor stages from the second column of clinicalfile    

    ## replace 0 (missing values) in tumor stage with nan
    choicelist = np.where(choicelist==0, np.nan, choicelist)

    # For every row
    for iCnt in range(np.shape(labelfile)[0]):
        condlist = clinicalfile[:, 0] == labelfile[iCnt, 0]##comparing the elements and storing the result in the condlist will be a Boolean array with True, false

        ## clinicalfile[:, 1]: This code extracts the entire second column (column with index 1) from the NumPy array clinicalfile
        ## choicelist contains the float number representations of tumor stages from the second column of clinicalfile
        # Update the last feature, by joining on ID
        mFeatures[iCnt, iFeatCount - 1] = np.select(condlist, choicelist)
        ##mFeatures[iCnt, iFeatCount - 1] iCnt is used as the row index and last column
        ##np.select will select values from choicelist based on the corresponding conditions in condlist
    vClass = labelfile[:, 1]
    sampleIDs = labelfile[:, 0]
    print("This is the vClass: ")
    print(vClass)
    # DEBUG LINES
    message("Found classes:\n%s" % (str(vClass)))
    message("Found sample IDs:\n%s" % (str(sampleIDs)))
    #############
    # DEBUG LINES
    # message("Found tumor types:\n%s" % (
    #     "\n".join(["%s:%s" % (x, y) for x, y in zip(labelfile[:, 0], mFeatures[:, iFeatCount - 1])])))
    #############
    message("Splitfeatures: This is the mFeatures...")
    message(mFeatures)
    message("Splitting features... Done.")

    return mFeatures, vClass, sampleIDs

In [67]:
# Find only the control samples
def getControlFeatureMatrix(mAllData, vLabels):
    """
    Gets the features of control samples only.
    :param mAllData: The full matrix of data (control plus tumor data).
    :param vLabels: The matrix of labels per case/instance.
    :return: The subset of the data matrix, reflecting only control cases/instances.
    """
    message("Finding only the control data...")
    choicelist = mAllData
    #--- condlist = isEqualToString(vLabels, 'Solid_Tissue_Normal')##epistrefei true, false se numpy array
    # 0 is the label for controls
    condlist = vLabels == "0"
    message("This is the control feature matrix:")
    print(choicelist[condlist])## ektupvnei osa einai true dld osa exoyn Solid_Tissue_Normal
    message("Data shape: %s" % (str(np.shape(choicelist[condlist]))))## epistrefei shape choicelist matrix
    message("Finding only the control data...Done")
    return choicelist[condlist]##epistrefei subset tou sunolikou matrix me mono osa exoun Solid_Tissue_Normal

In [13]:
def getNonControlFeatureMatrix(mAllData, vLabels):
    """
    Returns the subset of the feature matrix, corresponding to non-control (i.e. tumor) data.
    :param mAllData: The full feature matrix of case/instance data.
    :param vLabels: The label matrix, defining what instance is what type (control/tumor).
    :return: The subset of the feature matrix, corresponding to non-control (i.e. tumor) data
    """
    choicelist = mAllData
    condlist = vLabels == "1"
    return choicelist[condlist]

## Check for samples that don't have 3 omic levels
## Will be removed

In [32]:
def find_rows_with_nan(data):
    # Create a boolean mask indicating NaN values
    nan_mask = np.isnan(data)
    
    # Use np.all along axis 1 to check if all values in each row are True (indicating NaN)
    rows_with_nan = np.all(nan_mask, axis=1)
    
    # Get the indices of rows with NaN
    indices_of_rows_with_nan = np.where(rows_with_nan)[0]
    
    return indices_of_rows_with_nan


rows_with_nan_indices_mrna = find_rows_with_nan(mRNA)
print(rows_with_nan_indices_mrna)
rows_with_nan_indices_mirna = find_rows_with_nan(miRNA)
print(rows_with_nan_indices_mirna)
rows_with_nan_indices_methylation = find_rows_with_nan(methylation)
print(rows_with_nan_indices_methylation)

[ 39  44  53  57  66  73  76 140 185 363 386 423]
[ 35  39  44  53  57 252 371]
[46 61 73 76]


In [33]:
## find the ids that don't have data at all the levels
print(sampleIDs[rows_with_nan_indices_mrna])
print(sampleIDs[rows_with_nan_indices_mirna])
print(sampleIDs[rows_with_nan_indices_methylation])
test=[]
test.extend(rows_with_nan_indices_mirna)
test.extend(rows_with_nan_indices_methylation)
test.extend(rows_with_nan_indices_mrna)
print(sampleIDs[np.unique(test)])

NameError: name 'sampleIDs' is not defined

## END

In [34]:
def rows_filtering(input_matrix,nan_threshold=0.2):
    """
    It returns the filtered matrix for rows and an array with the rows that keep
    """
    ## length of row
    rows_length = input_matrix.shape[1]
    ## count nan per row
    nan_per_row = count_nan_per_row(input_matrix)
    ## compute the frequency of nan per row
    nan_frequency  = nan_per_row / rows_length
    ## return an array with boolean values, that show the rows with <=nan_threshold 
    rows_to_keep = nan_frequency <= nan_threshold
    ## filtering the matrix by rows_to_keep
    input_matrix = input_matrix[rows_to_keep, :]
    return input_matrix,rows_to_keep

def count_nan_per_row(input_matrix):
    nan_count_per_column = np.sum(np.isnan(input_matrix), axis=1)
    return nan_count_per_column

In [38]:
def columns_filtering(input_matrix,nan_threshold=0.2):
    """
    It returns the filtered matrix for columns and an array with the columns that keep
    """
    ## length of columns
    columns_length = input_matrix.shape[0]
    ## count nan per column
    nan_per_column = count_nan_per_column(input_matrix)
    ## compute the frequency of nan per column
    nan_frequency  = nan_per_column / columns_length
    ## return an array with boolean values, that show the columns with <=nan_threshold t
    columns_to_keep = nan_frequency <= nan_threshold
    ## filtering the matrix by columns_to_keep
    input_matrix = input_matrix[:, columns_to_keep]
    return input_matrix,columns_to_keep

def count_nan_per_column(input_matrix):
    nan_count_per_column = np.sum(np.isnan(input_matrix), axis=0)
    return nan_count_per_column

In [8]:
data=loadPatientAndControlData()

Loading features...
This is the datafile...
[[1.08013500e+02 0.00000000e+00 2.04832800e+02 ... 3.34862218e-01
  5.80957120e-01 4.18015401e-02]
 [1.86705600e+02 1.97600000e-01 1.60024300e+02 ... 3.43040715e-01
  5.68217306e-01 4.50390829e-02]
 [7.24201000e+01 4.11000000e-02 9.93833000e+01 ... 3.10183398e-01
  5.06576681e-01 4.39692210e-02]
 ...
 [3.13020000e+01 3.37000000e-02 1.01871000e+02 ... 3.40775401e-01
  5.23353440e-01 3.69211732e-02]
 [8.84820000e+00 0.00000000e+00 2.19054400e+02 ... 3.34662819e-01
  5.71726075e-01 2.54216861e-01]
 [4.93555000e+01 8.99000000e-02 8.18803000e+01 ... 3.43304371e-01
  5.72738669e-01 1.09697743e-01]]
Loading features... Done.


In [9]:
mRNA = data[:, :60660]
miRNA = data[:, 60660:62541]
methylation = data[:, 62541:]

In [ ]:
hist_for_cols(miRNA,"miRNA")
hist_for_rows(miRNA,"miRNA")

In [10]:
def hist_for_cols(data,level):
    freq = count_nan_per_column(data)
    # print(a)
    # Creating plot
    
    relative_freq = freq/np.shape(data)[0]
    # print(np.shape(data)[0])
    # print(b)
    # Creating plot
    fig = plt.figure(figsize =(10, 7))
    
    # Set the bins parameter to control the width of each column
    bins = np.arange(0, 1.1, 0.1)  # Adjust the range and step size as needed
    
    # Save the objects, in order to keep bars for bar_label
    counts, edges, bars = plt.hist(relative_freq, bins=bins) 
    ## plot labels over the bars
    plt.bar_label(bars)
    
    # Set x-axis limits (ensure minimum value is set to 0)
    plt.xlim(0, 1)
    
    # Set x-axis ticks every 0.1
    plt.xticks(np.arange(0, 1.1, 0.1))
    
    plt.title("Relative frequency histogram for features in "+level) 
    
    plt.xlabel('Relative frequency of NaN')
    plt.ylabel('Counts')
    # show plot
    plt.show()
    # fig.savefig(level+'relative_freq.png')

In [11]:
def hist_for_rows(data,level):
    freq = count_nan_per_row(data)
    # print(a)
    
    relative_freq = freq/np.shape(data)[1]
    # print(np.shape(data)[0])
    # print(b)
    # Creating plot
    fig = plt.figure(figsize =(10, 7))
    
    # Set the bins parameter to control the width of each column
    bins = np.arange(0, 1.1, 0.1)  # Adjust the range and step size as needed
    
    # Save the objects, in order to keep bars for bar_label
    counts, edges, bars = plt.hist(relative_freq, bins=bins) 
    ## plot labels over the bars
    plt.bar_label(bars)
    
    # Set x-axis limits (ensure minimum value is set to 0)
    plt.xlim(0, 1)
    
    # Set x-axis ticks every 0.1
    plt.xticks(np.arange(0, 1.1, 0.1))
    
    plt.title("Relative frequency histogram for patients in "+level) 

    plt.xlabel('Relative frequency of NaN')
    plt.ylabel('Counts')
    # show plot
    plt.show()
    # fig.savefig(level+'relative_freq_in_patients.png')

In [102]:
def postProcessFeatures(mFeatures, mControlFeatures):
    """
    Post-processes feature matrix to replace NaNs with control instance feature mean values, and also to remove
    all-NaN columns.

    :param mFeatures: The matrix to pre-process.
    :param mControlFeatures: The subset of the input matrix that reflects control instances.
    :return: The post-processed matrix, without NaNs.
    """
    message("Replacing NaNs from feature set...")
    # DEBUG LINES
    message("Data shape before replacement: %s" % (str(np.shape(mFeatures))))
    #############

    # WARNING: Imputer also throws away columns it does not like
    # imputer = Imputer(strategy="mean", missing_values="NaN", verbose=1)
    # mFeatures_noNaNs = imputer.fit_transform(mFeatures)

    # remove columns with only nan in control from all and control data 
    mFeatures, mControlFeatures = remove_empty_cols_from_control(mFeatures, mControlFeatures)
    
    # Extract means per control col
    # :-1 beccause the last column is about tunor stage
    mMeans = np.nanmean(mControlFeatures[:, :-1], axis=0)##calculate the mean of an array mControlFeatures along a specific axis
    # Find nans
    inds = np.where(np.isnan(mFeatures[:, :-1]))## Find nans
    # Do replacement
    mFeatures[inds] = np.take(mMeans, inds[1])## Do replacement

    message("Are there any NaNs after postProcessing?")
    message(np.any(np.isnan(mFeatures[:, :-1])))
    # DEBUG LINES
    message("Data shape after replacement: %s" % (str(np.shape(mFeatures))))
    #############

    # TODO: Check below
    # WARNING: If a control data feature was fully NaN, but the corresponding case data had only SOME NaN,
    # we would NOT successfully deal with the case data NaN, because there would be no mean to replace them by.

    #############
    message("Replacing NaNs from feature set... Done.")

    # Convert np array to panda dataframe
    arr = np.array(mFeatures)

    message("Removing features that have only NaN values...")
    # mask = np.all(np.isnan(mFeatures), axis=0)
	##mask variable will be a boolean array with True at the positions where all values in the corresponding columns of mFeatures are NaN, and False where there is at least one non-NaN value in that column
    # mFeatures = mFeatures[:, ~mask]
    ##remove columns from the mFeatures array where all values are NaN, based on the mask boolean array
    message("Number of features after removal: %s" % (str(np.shape(mFeatures))))
    # message(mFeatures)
    # message("Removing features that have only NaN values...Done")

    message("Are there any NaNs after postProcessing, except last column?")
    message(np.any(np.isnan(mFeatures[:, :-1])))

    message("This is mFeatures in postProcessing...")
    message(mFeatures)
    return mFeatures

In [37]:
def getFeatureNames():
    """
    Returns the names of the features of the data matrix, as a list.
    :return: The list of feature names.
    """
    message("Loading feature names...")
    fControl = open(FEATURE_VECTOR_FILENAME, "r")
    # Assuming fMatrix is the file path
    df = pd.read_csv(fControl, delimiter=' ')
    
    # Extract the column names
    column_names = df.columns

    ## keep the column names except last two with tumor stage and labels
    column_names = column_names[:-2]
    print(column_names)
    ##numpy.genfromtxt function to read data from a file. This function is commonly used to load data from text files into a NumPy array.
    ##dtype=np.dtype("float"): This sets the data type for the resulting NumPy array to float
    fControl.close()
    message("Loading feature names... Done.")

    return column_names

In [104]:
mFeatures, vClass, sampleIDs = initializeFeatureMatrices(bResetFiles=True, bPostProcessing=True, bNormalize=False, bNormalizeLog2Scale=False)

Opening files...
Trying to load saved data... Failed:
User requested file reset...
Trying to load saved data from txt...
Loading labels and ids...
This is the label file...
[['TCGA-2F-A9KO-01A' '1']
 ['TCGA-2F-A9KP-01A' '1']
 ['TCGA-2F-A9KQ-01A' '1']
 ['TCGA-2F-A9KR-01A' '1']
 ['TCGA-2F-A9KT-01A' '1']
 ['TCGA-2F-A9KW-01A' '1']
 ['TCGA-4Z-AA7M-01A' '1']
 ['TCGA-4Z-AA7N-01A' '1']
 ['TCGA-4Z-AA7O-01A' '1']
 ['TCGA-4Z-AA7Q-01A' '1']
 ['TCGA-4Z-AA7R-01A' '1']
 ['TCGA-4Z-AA7S-01A' '1']
 ['TCGA-4Z-AA7W-01A' '1']
 ['TCGA-4Z-AA7Y-01A' '1']
 ['TCGA-4Z-AA80-01A' '1']
 ['TCGA-4Z-AA81-01A' '1']
 ['TCGA-4Z-AA82-01A' '1']
 ['TCGA-4Z-AA83-01A' '1']
 ['TCGA-4Z-AA84-01A' '1']
 ['TCGA-4Z-AA86-01A' '1']
 ['TCGA-4Z-AA87-01A' '1']
 ['TCGA-4Z-AA89-01A' '1']
 ['TCGA-5N-A9KI-01A' '1']
 ['TCGA-5N-A9KM-01A' '1']
 ['TCGA-BL-A0C8-01A' '1']
 ['TCGA-BL-A0C8-01A-1' '1']
 ['TCGA-BL-A0C8-01B' '1']
 ['TCGA-BL-A13I-01A' '1']
 ['TCGA-BL-A13I-01A-1' '1']
 ['TCGA-BL-A13I-01B' '1']
 ['TCGA-BL-A13J-01A' '1']
 ['TCGA-BL-A13J-0

In [68]:
control_matrix = getControlFeatureMatrix(mFeatures, vClass)

Finding only the control data...
This is the control feature matrix:
[[2.68000000e+02 5.00000000e+00 8.49000000e+02 ... 5.73032181e-01
  8.57334573e-02 4.00000000e+00]
 [           nan            nan            nan ... 5.75177586e-01
  8.27838426e-02 2.00000000e+00]
 [2.27000000e+02 1.10000000e+01 7.21000000e+02 ... 5.82756281e-01
  8.41971990e-02 3.00000000e+00]
 ...
 [9.24600000e+03 6.00000000e+00 1.55600000e+03 ... 5.67039850e-01
  4.90431718e-02 2.00000000e+00]
 [4.37100000e+03 1.10000000e+01 1.57600000e+03 ... 5.58199376e-01
  4.69360668e-02 2.00000000e+00]
 [3.39000000e+03 5.00000000e+00 1.85700000e+03 ... 5.80928643e-01
  5.97484651e-02 3.00000000e+00]]
Data shape: (23, 100472)
Finding only the control data...Done


## remove_empty_cols_from_control is only to run postProcessFeatures
## probably will be removed

In [74]:
def remove_empty_cols_from_control(mFeatures, control_m):
    ## it removes the columns that have only nan in control from all data and control data
    # Check which elements are NaN for all the data
    nan_mask_all = np.isnan(mFeatures)
    
    # Check if all elements in each column are NaN
    columns_with_nan_all = np.all(nan_mask_all, axis=0)
    
    # Get the indices of columns with only NaN values
    column_indices_all = np.where(columns_with_nan_all)[0]
    
    print("Columns with only NaN values in all data:", len(column_indices_all))
    
    # Check which elements are NaN for control data
    nan_mask_control = np.isnan(control_m)
    
    # Check if all elements in each column are NaN
    columns_with_nan_control = np.all(nan_mask_control, axis=0)
    
    # Get the indices of columns with only NaN values
    column_indices_control = np.where(columns_with_nan_control)[0]
    
    print("Columns with only NaN values in control matrix:", len(column_indices_control))
    
    # find common columns with only nan between all data and cintrol 
    common_cols = np.intersect1d(column_indices_all, column_indices_control)
    print("The common elements are:", len(result))
    
    # find columns that have only nan in control but haven't in all data
    diff_cols = np.setdiff1d(column_indices_control, column_indices_all)
    print("The different elements are:", diff_cols)
    
    diff_cols_matrix = mFeatures[:, diff_cols]
    # print("The matrix with the columns from diff_cols:")
    # print(diff_cols_matrix)
    
    nan_diff_elements = count_nan_per_column(diff_cols_matrix)
    print("The percentage of NaN in the different element: ", (nan_diff_elements/np.shape(diff_cols_matrix)[0])*100)


    # Use boolean indexing to get the submatrix with columns removed
    mFeatures = mFeatures[:, ~np.isin(np.arange(mFeatures.shape[1]), column_indices_control)]
    control_m = control_m[:, ~np.isin(np.arange(control_m.shape[1]), column_indices_control)]
    return mFeatures, control_m